In [5]:
import pandas as pd

In [6]:
df = pd.read_excel(r'C:\Users\barsuraj1\Desktop\Product Recommendation\datasets\Online Retail.xlsx')

In [7]:
df.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom

Goal is to recommend products purchased together by checing the frequency by which different items are purchased together, so we only need information about individual orders and individual products. 
as it will be more convenient for display purposes we will also use the Description. 
so we don't required the rest of the attributed.

We will keep two DataFrames.

One for Building the recommendation system with the following features:

InvoiceNo
StockCode

And one for matching the description to the StockCode:

StockCode
Description

In [8]:
# Modify StockCode to always be a string by prepending '_'
df['StockCode'] = df['StockCode'].apply(lambda x: '_'+str(x))

In [9]:
# DataFrame for building the recommendation system
orders = df[['InvoiceNo', 'StockCode']]
orders.head()

InvoiceNo StockCode
0    536365   _85123A
1    536365    _71053
2    536365   _84406B
3    536365   _84029G
4    536365   _84029E

In [10]:
# DataFrame for retrieving product descriptions
products = df[['StockCode', 'Description']].copy()
products.head()

StockCode                          Description
0   _85123A   WHITE HANGING HEART T-LIGHT HOLDER
1    _71053                  WHITE METAL LANTERN
2   _84406B       CREAM CUPID HEARTS COAT HANGER
3   _84029G  KNITTED UNION FLAG HOT WATER BOTTLE
4   _84029E       RED WOOLLY HOTTIE WHITE HEART.

In [11]:
# Drop duplicated products
products = products[~products.duplicated()]
products.head()

StockCode                          Description
0   _85123A   WHITE HANGING HEART T-LIGHT HOLDER
1    _71053                  WHITE METAL LANTERN
2   _84406B       CREAM CUPID HEARTS COAT HANGER
3   _84029G  KNITTED UNION FLAG HOT WATER BOTTLE
4   _84029E       RED WOOLLY HOTTIE WHITE HEART.

In [12]:
# Drop descriptions that are not uppercase
products = products[products['Description'].str.upper() == products['Description']]

In [13]:
# Keep only the first Description of each product
products = products[~products.duplicated(subset=['StockCode'])]

products

StockCode                          Description
0        _85123A   WHITE HANGING HEART T-LIGHT HOLDER
1         _71053                  WHITE METAL LANTERN
2        _84406B       CREAM CUPID HEARTS COAT HANGER
3        _84029G  KNITTED UNION FLAG HOT WATER BOTTLE
4        _84029E       RED WOOLLY HOTTIE WHITE HEART.
...          ...                                  ...
509369   _85179a              GREEN BITTY LIGHT CHAIN
512588    _23617  SET 10 CARDS SWIRLY XMAS TREE 17104
527065   _90214U            LETTER "U" BLING KEY RING
537224   _47591b             SCOTTIES CHILDRENS APRON
540421    _23843          PAPER CRAFT , LITTLE BIRDIE

[3905 rows x 2 columns]

In [14]:
products.shape

(3905, 2)

In [15]:
# Set the index to StockCode
products = products.set_index('StockCode')

In [16]:
# Convert to Series for eve easier lookups
products = products['Description']

In [17]:
# Test it out
products['_71053']

'WHITE METAL LANTERN'

In [18]:
# No of unique products
len(products)

3905

In [19]:
### no of unique products
products.nunique()

3779

### Restructure the data

We would like each Invoice Number to give us a list of stock codes.

In [20]:
def string_list(x):
    return [str(i) for i in x]

In [21]:
orders = orders.groupby('InvoiceNo')['StockCode'].apply(list).reset_index()
orders.head()

InvoiceNo                                          StockCode
0    536365  [_85123A, _71053, _84406B, _84029G, _84029E, _...
1    536366                                   [_22633, _22632]
2    536367  [_84879, _22745, _22748, _22749, _22310, _8496...
3    536368                   [_22960, _22913, _22912, _22914]
4    536369                                           [_21756]

In [22]:
from mlxtend.preprocessing import TransactionEncoder

In [23]:
te = TransactionEncoder()

te.fit(orders['StockCode'])
orders_1hot = te.transform(orders['StockCode'])

In [24]:
orders_1hot = pd.DataFrame(orders_1hot, columns =te.columns_)
orders_1hot.head()

_10002  _10080  _10120  _10123C  _10123G  _10124A  _10124G  _10125  _10133  \
0   False   False   False    False    False    False    False   False   False   
1   False   False   False    False    False    False    False   False   False   
2   False   False   False    False    False    False    False   False   False   
3   False   False   False    False    False    False    False   False   False   
4   False   False   False    False    False    False    False   False   False   

   _10134  ...     _M  _PADS  _POST     _S  _gift_0001_10  _gift_0001_20  \
0   False  ...  False  False  False  False          False          False   
1   False  ...  False  False  False  False          False          False   
2   False  ...  False  False  False  False          False          False   
3   False  ...  False  False  False  False          False          False   
4   False  ...  False  False  False  False          False          False   

   _gift_0001_30  _gift_0001_40  _gift_0001_50     _m  
0          False          False          False  False  
1          False          False          False  False  
2          False          False          False  False  
3          False          False          False  False  
4          False          False          False  False  

[5 rows x 4070 columns]

### Apriori

In [25]:
from mlxtend.frequent_patterns import apriori

In [26]:
%%timeit -n1 -r1

apriori(orders_1hot, min_support=0.01, max_len=2, use_colnames=True)

5min 7s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [27]:
is_ap = apriori(orders_1hot, min_support=0.01, max_len=2, use_colnames=True)

In [33]:
is_ap.head() 

support    itemsets
0  0.020193    (_15036)
1  0.012587  (_15056BL)
2  0.017876   (_15056N)
3  0.011236    (_16237)
4  0.012510    (_20675)

## FP Growth

In [34]:
from mlxtend.frequent_patterns import fpgrowth

In [35]:
%%timeit -n1 -r1

fpgrowth(orders_1hot, min_support=0.01, max_len=2, use_colnames=True)

10.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [36]:
is_fp = fpgrowth(orders_1hot, min_support=0.01, max_len=2, use_colnames=True)

In [37]:
is_fp.head()

support   itemsets
0  0.086718  (_85123A)
1  0.017915  (_84029G)
2  0.016911  (_84029E)
3  0.014865   (_22752)
4  0.013205   (_71053)

## Show that is_ap and is_fp both contain the same itemsets

In [38]:
def itemset_to_ordered_string(itemset):
    return ','.join(sorted(list(itemset)))

In [39]:
ap_itemset_strings = is_ap['itemsets'].apply(itemset_to_ordered_string)

In [40]:
ap_itemset_strings = ap_itemset_strings.sort_values().reset_index(drop=True)

In [41]:
ap_itemset_strings.head()

0      _15036
1    _15056BL
2     _15056N
3      _16237
4      _20675
Name: itemsets, dtype: object

In [42]:
fp_itemset_strings = is_fp['itemsets'].apply(itemset_to_ordered_string)

In [43]:
fp_itemset_strings = fp_itemset_strings.sort_values().reset_index(drop=True)
fp_itemset_strings.head()

0      _15036
1    _15056BL
2     _15056N
3      _16237
4      _20675
Name: itemsets, dtype: object

In [44]:
# test to see if the itemset lists are equal
fp_itemset_strings.equals(ap_itemset_strings)

True

## Calculate Association Rules

In [45]:
from mlxtend.frequent_patterns import association_rules

In [46]:
is_fp

support          itemsets
0     0.086718         (_85123A)
1     0.017915         (_84029G)
2     0.016911         (_84029E)
3     0.014865          (_22752)
4     0.013205          (_71053)
...        ...               ...
997   0.010077  (_23344, _23203)
998   0.010039  (_23344, _22086)
999   0.011853  (_23295, _23293)
1000  0.010077  (_23296, _23293)
1001  0.010463  (_22112, _23355)

[1002 rows x 2 columns]

In [47]:
rules = association_rules(is_fp, metric="lift", min_threshold=10)
rules.head()

antecedents consequents  antecedent support  consequent support   support  \
0    (_21755)    (_21754)            0.024363            0.030386  0.011467   
1    (_21754)    (_21755)            0.030386            0.024363  0.011467   
2    (_22748)    (_22745)            0.016988            0.016448  0.012124   
3    (_22745)    (_22748)            0.016448            0.016988  0.012124   
4    (_22727)    (_22726)            0.041737            0.038726  0.024942   

   confidence       lift  leverage  conviction  
0    0.470681  15.490025  0.010727    1.831815  
1    0.377382  15.490025  0.010727    1.566993  
2    0.713636  43.387751  0.011844    3.434626  
3    0.737089  43.387751  0.011844    3.738955  
4    0.597595  15.431412  0.023326    2.388821

In [48]:
rules.shape

(280, 9)

##  Predictions

In [49]:
def predict(antecedent, rules, max_results= 6):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a messy way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results].reset_index(drop=True)

In [50]:
preds = predict({'_20712'}, rules)

In [51]:
preds

0    _22379
1    _20713
2    _21930
3    _21931
4    _22386
5    _21928
Name: consequents, dtype: object

In [52]:
print(products['_20712'])

JUMBO BAG WOODLAND ANIMALS


In [53]:
for stockid in preds:  
    print(products[stockid])

RECYCLING BAG RETROSPOT 
JUMBO BAG OWLS
JUMBO STORAGE BAG SKULLS
JUMBO STORAGE BAG SUKI
JUMBO BAG PINK POLKADOT
JUMBO BAG SCANDINAVIAN PAISLEY


In [54]:
print(products['_21755'])

LOVE BUILDING BLOCK WORD


In [55]:
# get the predictions
preds = predict({'_21755'}, rules)

# Display the descriptions of the predictions
for stockid in preds:  
    print(products[stockid])

HOME BUILDING BLOCK WORD


In [56]:
print(products['_22748'])

POPPY'S PLAYHOUSE KITCHEN


In [57]:
# get the predictions
preds = predict({'_22748'}, rules)

# Display the descriptions of the predictions
for stockid in preds:  
    print(products[stockid])

POPPY'S PLAYHOUSE BEDROOM 
POPPY'S PLAYHOUSE LIVINGROOM 


In [58]:
rules.antecedents

0      (_21755)
1      (_21754)
2      (_22748)
3      (_22745)
4      (_22727)
         ...   
275    (_23293)
276    (_23296)
277    (_23293)
278    (_22112)
279    (_23355)
Name: antecedents, Length: 280, dtype: object

In [77]:
rules.antecedents[21]

frozenset({'_22835'})

In [80]:
print(products['_22727'])

ALARM CLOCK BAKELIKE RED 


In [81]:
# get the predictions
preds = predict({'_22727'}, rules)

# Display the descriptions of the predictions
for stockid in preds:  
    print(products[stockid])

ALARM CLOCK BAKELIKE GREEN
ALARM CLOCK BAKELIKE PINK
ALARM CLOCK BAKELIKE IVORY
ALARM CLOCK BAKELIKE ORANGE
ALARM CLOCK BAKELIKE CHOCOLATE


In [62]:
rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         280 non-null    object 
 1   consequents         280 non-null    object 
 2   antecedent support  280 non-null    float64
 3   consequent support  280 non-null    float64
 4   support             280 non-null    float64
 5   confidence          280 non-null    float64
 6   lift                280 non-null    float64
 7   leverage            280 non-null    float64
 8   conviction          280 non-null    float64
dtypes: float64(7), object(2)
memory usage: 19.8+ KB


In [63]:
products.describe()

count                               3905
unique                              3779
top       METAL SIGN,CUPCAKE SINGLE HOOK
freq                                   6
Name: Description, dtype: object

In [69]:
products

StockCode
_85123A     WHITE HANGING HEART T-LIGHT HOLDER
_71053                     WHITE METAL LANTERN
_84406B         CREAM CUPID HEARTS COAT HANGER
_84029G    KNITTED UNION FLAG HOT WATER BOTTLE
_84029E         RED WOOLLY HOTTIE WHITE HEART.
                          ...                 
_85179a                GREEN BITTY LIGHT CHAIN
_23617     SET 10 CARDS SWIRLY XMAS TREE 17104
_90214U              LETTER "U" BLING KEY RING
_47591b               SCOTTIES CHILDRENS APRON
_23843             PAPER CRAFT , LITTLE BIRDIE
Name: Description, Length: 3905, dtype: object

In [70]:
rules

antecedents consequents  antecedent support  consequent support   support  \
0      (_21755)    (_21754)            0.024363            0.030386  0.011467   
1      (_21754)    (_21755)            0.030386            0.024363  0.011467   
2      (_22748)    (_22745)            0.016988            0.016448  0.012124   
3      (_22745)    (_22748)            0.016448            0.016988  0.012124   
4      (_22727)    (_22726)            0.041737            0.038726  0.024942   
..          ...         ...                 ...                 ...       ...   
275    (_23293)    (_23295)            0.021506            0.017336  0.011853   
276    (_23296)    (_23293)            0.014826            0.021506  0.010077   
277    (_23293)    (_23296)            0.021506            0.014826  0.010077   
278    (_22112)    (_23355)            0.033436            0.030502  0.010463   
279    (_23355)    (_22112)            0.030502            0.033436  0.010463   

     confidence       lift  leverage  conviction  
0      0.470681  15.490025  0.010727    1.831815  
1      0.377382  15.490025  0.010727    1.566993  
2      0.713636  43.387751  0.011844    3.434626  
3      0.737089  43.387751  0.011844    3.738955  
4      0.597595  15.431412  0.023326    2.388821  
..          ...        ...       ...         ...  
275    0.551167  31.793373  0.011480    2.189376  
276    0.679688  31.604859  0.009758    3.054811  
277    0.468582  31.604859  0.009758    1.853857  
278    0.312933  10.259450  0.009443    1.411068  
279    0.343038  10.259450  0.009443    1.471263  

[280 rows x 9 columns]

In [73]:
rules.to_csv(r"C:\Users\barsuraj1\Desktop\Product Recommendation\rules_tab.csv")

In [74]:
products.to_csv(r"C:\Users\barsuraj1\Desktop\Product Recommendation\products_tab.csv")